In [ ]:
!pip install --force-reinstall transformers accelerate

In [ ]:
!pip install rouge_score

In [ ]:
my_token = ''
from huggingface_hub import login
login()

In [ ]:
!pip install datasets sentencepiece

In [ ]:
import transformers
from datasets import load_dataset

from tokenizers import Tokenizer
from transformers import (AutoTokenizer,
                          PreTrainedTokenizer,
                          AutoModelForSeq2SeqLM,
                          DataCollatorForSeq2Seq,
                          Seq2SeqTrainingArguments,
                          Seq2SeqTrainer
)



# model name
AraBART = "moussaKam/AraBART"
# dataset name
data = "csebuetnlp/xlsum"
# transformer version
print(transformers.__version__)

4.34.1


In [ ]:
dataset = load_dataset( "csebuetnlp/xlsum" , "arabic")
dataset

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'url', 'title', 'summary', 'text'],
        num_rows: 37519
    })
    test: Dataset({
        features: ['id', 'url', 'title', 'summary', 'text'],
        num_rows: 4689
    })
    validation: Dataset({
        features: ['id', 'url', 'title', 'summary', 'text'],
        num_rows: 4689
    })
})

In [ ]:
from datasets import load_dataset
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/AIC/data_csv.csv')
print(df.head)
raw_dataset = load_dataset('csv', data_files='/content/drive/MyDrive/AIC/data_csv.csv')

<bound method NDFrame.head of      Unnamed: 0                                               text  \
0             0  وتحت عنوان من الكارثة إلى التحدى يبدأ الكاتب ع...   
1             1  ولم يعترف دبلوماسيو هاتين الدولتين بالعريضة ال...   
2             2  قامت ولاية حلب بعد اعلان الجنرال الفرنسي هنري ...   
3             3  دولة مصر العربيه هي ليست اي دوله وليست اي شعب ...   
4             4  السوريون يصرون على استقلال بلادهم : و مثلما رف...   
..          ...                                                ...   
149         149  حزب الوفد سيحتفل بمئوية ثورة 1919 يوم 9 مارس ا...   
150         150  حيث أعلن مجلس قيادة الثورة في 18 يونيه 1953 قي...   
151         151  وبرغم أن عبد الرحمن فهمي كان يضم في ذلك الجهاز...   
152         152  ولم تقتصر مقومات بورسعيد كمدينة عالمية منذ نشأ...   
153         153  أول رئيس للجزائر بعد الاستقلال الرئيس أحمد بن ...   

                                               summary  
0    يبدأ الكاتب عرض الكتاب الرابع تحت عنوان من الك...  
1    دبلوماسيو 

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
split = raw_dataset['train'].train_test_split(test_size=0.2, seed=42)
print(split)

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'text', 'summary'],
        num_rows: 123
    })
    test: Dataset({
        features: ['Unnamed: 0', 'text', 'summary'],
        num_rows: 31
    })
})


In [ ]:
tokenizer = AutoTokenizer.from_pretrained( AraBART )
traind_model = AutoModelForSeq2SeqLM.from_pretrained( AraBART )

In [ ]:
max_input_length = 1024
max_target_length = 256

def preprocessing(rows):
    inputs = [row for row in rows["text"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(rows["summary"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
split = split.map(preprocessing, batched=True)
dataset = dataset.map(preprocessing, batched=True)

Map:   0%|          | 0/123 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3848: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/31 [00:00<?, ? examples/s]

Map:   0%|          | 0/37519 [00:00<?, ? examples/s]

Map:   0%|          | 0/4689 [00:00<?, ? examples/s]

Map:   0%|          | 0/4689 [00:00<?, ? examples/s]

In [ ]:
########################################
tokenizer = AutoTokenizer.from_pretrained( '/content/drive/MyDrive/AIC/bertModel_1_epoch' )
traind_model = AutoModelForSeq2SeqLM.from_pretrained( '/content/drive/MyDrive/AIC/bertModel_1_epoch' )

##################################

In [ ]:
batch_size = 1
arguments = Seq2SeqTrainingArguments(
    "/content/drive/MyDrive/AIC/arabartsummarization",
    evaluation_strategy = "epoch",
    learning_rate = 5e-7,
    per_device_train_batch_size = batch_size,
    per_device_eval_batch_size = batch_size,
    num_train_epochs=1,
    save_steps=4000,
    # push_to_hub=True,

    predict_with_generate=True,
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=traind_model)
trainer = Seq2SeqTrainer(
    traind_model,
    arguments,
    train_dataset = split["train"],
    eval_dataset = split["test"],
    data_collator = data_collator,
    tokenizer = tokenizer,
)

trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,2.992317


TrainOutput(global_step=123, training_loss=3.167949397389482, metrics={'train_runtime': 17.2142, 'train_samples_per_second': 7.145, 'train_steps_per_second': 7.145, 'total_flos': 29061629890560.0, 'train_loss': 3.167949397389482, 'epoch': 1.0})

In [ ]:
# trainer.save_model('/content/drive/MyDrive/AIC/bertModel_1_epoch')

In [ ]:
from IPython.display import display, HTML

# Configure output behavior
display(HTML("<style>pre { white-space: pre-wrap; }</style>"))
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu"),

generated_output = traind_model.generate(
    input_ids=tokenizer.encode(df['text'][0], return_tensors="pt"),
    max_length=256,  # maximum length of the generated sequence
    num_beams=5,
    early_stopping=True
)

print("\nGenerated text>>>>>>>>>>>>>>>>>>>>>>>>>>>>:\n")
# Decode the generated output
generated_text = tokenizer.decode(generated_output[0], skip_special_tokens=True)
print(generated_text)
print('\noriginal text>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>:\n',df['text'][0])



Generated text>>>>>>>>>>>>>>>>>>>>>>>>>>>>:

يبدأ الكاتب عرض الكتاب الرابع عن حرب أكتوبر، حيث يوضح كيف كانت إسرائيل فرحة بنصرها عام 67 وأنها ارتاحت لاعتقادها بأن هناك وقتا طويلا وطويلا جدا قبل أن يفيق العرب من صدمة 67، وكيف أن القوات الجوية للجمهورية العربية المتحدة فاجأتها بعد شهر واحد من نهاية حرب 67 بهجوم جوى عنيف على مواقعها فى سيناء وكان هذا إعلانا عن بداية حرب من نوع جديد هى حرب الاستنزاف التى استمرت حتى تم وقف إطلاق النار بين الطرفين فى 8 أغسطس 1970 ثم وفاة عبدالناصر وتولى أنور السادات حكم مصر واستعداده للحرب، ثم بداية الضغوط السياسية على الرئيس أنور السادات من 17 19 أكتوبر ثم تطورات هذه المعركة.

original text>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>:
 وتحت عنوان من الكارثة إلى التحدى يبدأ الكاتب عرض الكتاب الرابع ، حيث يوضح كيف كانت إسرائيل فرحة بنصرها عام 67 وأنها ارتاحت لاعتقادها بأن هناك وقتا طويلا وطويلا جدا قبل أن يفيق العرب من صدمة 67، وكيف أن القوات الجوية للجمهورية العربية المتحدة قد فاجأتها بعد شهر واحد من نهاية حرب 67 بهجوم جوى عنيف على مواقعها فى سيناء وكان هذا إعلانا عن بد

In [ ]:
generated = []
target = []
for i in range(10):

  generated_output = traind_model.generate(
    input_ids=tokenizer.encode(df['text'][i], return_tensors="pt").to(0),max_length=256,  num_beams=5,  early_stopping=True  )
  generated_text = tokenizer.decode(generated_output[0], skip_special_tokens=True)
  generated.append(generated_text)
  target.append(df['summary'][i])

In [ ]:

from datasets import load_metric

rouge = load_metric("rouge")

target_text = df['summary'][0]
print(generated_text)

scores = rouge.compute(predictions=[generated], references=[target])

print(scores)

عرض الكتاب الرابع عن حرب أكتوبر وكيف كانت إسرائيل فرحة بنصرها عام 67 وكيف أن القوات الجوية للجمهورية العربية المتحدة فاجأتها بعد شهر واحد من نهاية حرب الاستنزاف التى استمرت حتى تم وقف إطلاق النار بين الطرفين فى 8 أغسطس 1970، ثم وفاة عبدالناصر وتولى أنور السادات حكم مصر، ويعرض الكاتب للخطط والاستعدادات المصرية ثم الاستعدادات الإسرائيلية، ويتوقف الكاتب عند يوم 8 أكتوبر، ويقول: إن هذا اليوم كان أسوأ هزيمة فى تاريخ الجيش الإسرائيلى، ثم يعود ثانية إلى الجبهة السورية، ثم يعرض للثغرة أو ما عرف بعملية المزرعة الصينية يوم 16 و 15 أكتوبر، ويستعرض الكاتب المعركة الخاصة بالاستيلاء على مدينة السويس من 23 أكتوبر إلى 25 أكتوبر، وكيف أن الملك حسين قرر دخول الحرب ضد إسرائيل يوم 9 أكتوبر.
{'rouge1': AggregateScore(low=Score(precision=0.3181818181818182, recall=0.3684210526315789, fmeasure=0.3414634146341463), mid=Score(precision=0.3181818181818182, recall=0.3684210526315789, fmeasure=0.3414634146341463), high=Score(precision=0.3181818181818182, recall=0.3684210526315789, fmeasure=0.3414634146341463)), '

In [ ]:
def sumar(text):

  generated_output = traind_model.generate(
    input_ids=tokenizer.encode(text, return_tensors="pt").to(0),
    max_length=256,  # maximum length of the generated sequence
    num_beams=5,
    early_stopping=True
    )

  generated_text = tokenizer.decode(generated_output[0], skip_special_tokens=True)
  return generated_text

In [ ]:
import json

data = []
with open('/content/drive/MyDrive/AIC/validation_data.jsonl') as f:

    for line in f:
        data.append(json.loads(line))

In [ ]:
from tqdm import tqdm
from tqdm import tqdm
with open('/content/results.jsonl', 'w',encoding='utf-8') as outfile:

    for pr in tqdm(data):
        id = pr['example_id']
        text = pr['paragraph']
        summary = sumar(text)
        line = {'example_id':id,'summary':summary}
        json.dump(line,outfile, ensure_ascii=False)
        outfile.write('\n')

100%|██████████| 272/272 [07:22<00:00,  1.63s/it]


In [ ]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 5000})'''))


<IPython.core.display.Javascript object>